### This is my hw4 - Task 1: Affect in Tweets
#### Tatiana Shavrina

More specificly, I used the data from EI-reg for English
#### Task EI-reg: Detecting Emotion Intensity (regression)

#### The task is formulated:
determine the intensity or amount of X felt by the speaker—a real-valued score between 0 and 1:
a score of 1: speaker feels the highest amount of X
a score of 0: speaker feels the lowest amount of X

#### Data:
for each tweet
a tweet
an affect category X (anger, fear, joy, or sadness)

I am using training data as a training data, and developers data provided - as a test set
I've put the pandas-transformed data into archive with password, you can find it here:

#### Compatability:
if you are using Linux, change all the filepath slaches from '\\' to '/'

In [1]:
# -*- coding: utf-8 -*- 

import pandas as pd
import os, re
import numpy as np
from sklearn.utils import shuffle

In [4]:
def txt_to_dataframe(lst):
    textid,text,emo,weight=[],[],[],[]
    for filename in lst:
        file = open(filename, 'r', encoding='utf8').readlines()
        textid = textid+[line.split('\t')[0] for line in file if '\t' in line]
        text = text+[line.split('\t')[1] for line in file if '\t' in line]
        emo = emo+[line.split('\t')[2] for line in file if '\t' in line]
        weight = weight+[line.split('\t')[3].strip('\n') for line in file if '\t' in line]
    data = pd.DataFrame(
    {'textid': textid,
     'text': text,
     'emo': emo,
     'weight': weight
    })
    
    data = shuffle(data)
    return data

In [7]:
train,test = [],[]
workdir = r'C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train'
for file in os.listdir(workdir):
    if '2018-EI-reg' in file:
        if "dev" in file:
            test.append(workdir+'\\'+file)
        else:
            train.append(workdir+'\\'+file)
test, train = txt_to_dataframe(test),txt_to_dataframe(train)

In [8]:
test.head()

,emo,text,textid,weight
426,joy,I finally left the dark room I've been crying ...,2018-en-joy-dev-27,0.242
657,sadness,"If you are going to be a nurse, learn to be a ...",2018-en-sadness-dev-58,0.413
762,sadness,Chopper &lt;&lt;Dammit! Just sink already!&gt;...,2018-en-sadness-dev-163,0.304
121,anger,@mickyrivera29 interesting but a transfer too ...,2018-en-anger-dev-122,0.578
211,fear,I'm back on Twitter! #madden #madden18 #madden...,2018-en-fear-dev-12,0.125


In [9]:
train.head()

,emo,text,textid,weight


In [10]:
train.emo.value_counts()

Series([], Name: emo, dtype: int64)

In [11]:
test.to_csv(path_or_buf=r'C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\test.csv', sep='\t',encoding='utf8')
train.to_csv(path_or_buf=r'C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\train.csv', sep='\t',encoding='utf8')

#### On our data, we can see that the classification is not multiclass, while hashtags show  the opposite 

10003	so ef whichever butt wipe pulled the fire alarm in davis bc I was sound asleep #pissed #angry #upset #tired #sad #tired #hangry ######	anger	0.896

####  imho, this is  a serious reason for the deterioration of classification quality, because this way  the data is not clean itself. Although the data is never clean :)


### Let's build a classifier on that data

In [12]:
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, Ridge, LinearRegression, LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### First, we will assume, that in the test data we already have emotional class - fear, anger , joy or sadness - because they are obviously got from hashtags, and we have these hashtags in the tweets. So we should train 4 classifiers for each emotion to predict the affect from 0 to 1 in a given emotion.
#### This is easy to do with regression analysis on text features - let's try it

In [13]:
a = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,5), analyzer='word', max_features=20000)),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    #('reducer', TruncatedSVD(n_components=Val3)),
    ('clf', Ridge(fit_intercept=False, random_state = 0))])
b = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,5), analyzer='word', max_features=20000)),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    #('reducer', TruncatedSVD(n_components=Val3)),
    ('clf', LinearRegression(n_jobs =-1))])

In [14]:
def traintestsplit(train, test, emot):
    X_train = [train.text.iloc[i] for i in range (len(train)) if train.emo.iloc[i]==emot]
    y_train = [float(train.weight.iloc[i]) for i in range (len(train)) if train.emo.iloc[i]==emot]
    X_test = [test.text.iloc[i] for i in range (len(test)) if test.emo.iloc[i]==emot]
    y_test = [float(test.weight.iloc[i]) for i in range (len(test)) if test.emo.iloc[i]==emot]
    return X_train, y_train, X_test, y_test

#### To test the obtained quality we will use evaluation script from
https://github.com/felipebravom/EmoInt

I slightly changed the code as it is suitable for python 2, and I am using python3 + I changed the output format
See https://github.com/TatianaShavrina/hselessons/blob/master/AffectTweets.rar

In [15]:
for emot in set(train.emo.values):
    outfile = open(workdir+'\\'+emot+'-pred.txt', 'w', encoding='utf8')
    testfile = open(workdir+'\\'+emot+'-gold.txt', 'w', encoding='utf8')
    X_train, y_train, X_test, y_test = traintestsplit(train, test, emot)
    a.fit(X_train, y_train)
    b.fit(X_train, y_train)
    predictions_a = [round(y,3) for y in a.predict(X_test)]
    predictions_b = [round(y,3) for y in b.predict(X_test)]
    
    outfile.close()
    testfile.close()
    print('linear regression on words')
    print('predicting '+emot)
    print("Mean absolute error regression loss ", str(mean_absolute_error(y_test, predictions_b)))
    print("Mean squared error regression loss ", str(mean_squared_error(y_test, predictions_b)))
    print("Mean squared logarithmic error regression loss ", str(mean_squared_log_error(y_test, predictions_b)))
    print("Median absolute error regression loss ",str(median_absolute_error(y_test, predictions_b)))
    print("r2_score", str(r2_score(y_test, predictions_b)))
    print()
    print('Ridge regression on words')
    print('predicting '+emot)
    print("Mean absolute error regression loss ", str(mean_absolute_error(y_test, predictions_a)))
    print("Mean squared error regression loss ", str(mean_squared_error(y_test, predictions_a)))
    print("Mean squared logarithmic error regression loss ", str(mean_squared_log_error(y_test, predictions_a)))
    print("Median absolute error regression loss ",str(median_absolute_error(y_test, predictions_a)))
    print("r2_score", str(r2_score(y_test, predictions_a)))
    print()



### Ridge regression seems a lot better! Let's save the model so that it could be reproducible

In [16]:
modelname = workdir +'\\ridge.pkl'
joblib.dump(a, modelname)

['C:\\Users\\Shavrina_T\\Documents\\other\\EI-reg-English-Train\\ridge.pkl']

In [17]:
for emot in set(train.emo.values):
    outfile = open(workdir+'\\'+emot+'-pred.txt', 'w', encoding='utf8')
    testfile = open(workdir+'\\'+emot+'-gold.txt', 'w', encoding='utf8')
    X_train, y_train, X_test, y_test = traintestsplit(train, test, emot)
    a.fit(X_train, y_train)
    for i in range(len(test)):
        ls1=[test.textid.iloc[i],test.text.iloc[i],test.emo.iloc[i]]
        ls2=[test.textid.iloc[i],test.text.iloc[i],test.emo.iloc[i],test.weight.iloc[i]]
        
        e = a.predict([test.text.iloc[i]])
        ls1.append(str(round(e[0],3)))
        outfile.write('\t'.join(ls1)+'\n')
        testfile.write('\t'.join(ls2)+'\n')
    outfile.close()
    testfile.close()



In [22]:
#! evaluate.py 1 anger-pred.txt anger-pred.txt
!python evaluate.py 4 C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\anger-pred.txt C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\anger-gold.txt C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\fear-pred.txt C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\fear-gold.txt C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\joy-pred.txt C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\joy-gold.txt C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\sadness-pred.txt C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\sadness-gold.txt

Pearson correlation between C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\anger-pred.txt and C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\anger-gold.txt:	0.0628256437243
Spearman correlation between C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\anger-pred.txt and C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\anger-gold.txt:	0.0682795805942
Pearson correlation for gold scores in range 0.5-1 between C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\anger-pred.txt and C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\anger-gold.txt:	0.0637663047328
Spearman correlation for gold scores in range 0.5-1 between C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\anger-pred.txt and C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\anger-gold.txt:	0.0558755151785
Pearson correlation between C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\fear-pred.txt and C:\Users\Shavrina_T\Documents\other\EI-reg-English-Train\fear